# Capstone Project
---
The focus of this project is to find the best neighbourhood in Toronto west. It will utilize location data from foursquare and machine learning to predict the most suitable location.


In [1]:
# import of standard libraries

import pandas as pd
import numpy as np

In [2]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!
